# Klassen für das Erstellen von Modellen eines Brettspiels

Es gibt eine generalisierte Klasse *Model*, die für alle 1 gegen 1 Brettspiele mit wertebasiertem bestärkendem Lernmodell dient. Das Lernen wird mittels neuronalen Netzen und nach der Logik des Q-Algorithmus umgesetzt.

Es werden 2 wesentliche Lernvarianten unterstützt:

* Online Training
* Offline Training

Unter *Online* wird das direkte Lernen pro Zustand verstanden. Nach jedem Zug wird der aktuelle Wert antrainiert und im Model gespeichert.
Unter *Offline* wird das Lernen von einer vorgegebenen Größe an gespeicherten Zuständen verstanden. Dabei wird erst eine Anzahl an Zügen gespielt. Danach wird das Spiel gestoppt und das Modell wird mit den eben gespielten Zuständen angelernt.

In [ ]:
from abc import abstractmethod
from pathlib import Path
import os
import numpy as np
import keras.models as Km
import keras.layers as kl
import random

from build.Board import Board

In [ ]:
class Model:
    """
    Model class for all 2 player based games with neural network training
    """

    def __init__(self, tag, online=False):
        """
        :param tag: used tag for neural network model (e.g. 1 for first player and -1 for second)
        """
        self.tag = tag
        self.epsilon = 0.1
        self.alpha = 0.5
        self.gamma = 1
        self.model = self.__load_model()
        self.history = []
        self.memory = []
        self.count_memory = 0
        self.batch_size = 10
        self.online = online

    def __load_model(self):
        """
        Loads previously saved model
        :return: loaded model
        """
        tag = '_first' if self.tag == 1 else '_second'

        if self.online:
            tag += '_online'

        s = 'model_values' + tag + '.h5'
        model_file = Path(s)

        if model_file.is_file():
            print('load model')
            model = Km.load_model(s)
            print('load model: ' + s)
        else:
            model = self.create_model()
        return model

    @abstractmethod
    def create_model(self):
        """
        Create new model with appropriate number of layers and network structure
        :return: created model
        """
        pass

    @abstractmethod
    def state_to_tensor(self, state, move):
        """
        Creates a tensor (2 dim array) based on a state and a move as input vector for nn
        :param state: current state
        :param move: current move
        :return: created tensor
        """
        pass

    @abstractmethod
    def one_hot_encode_state(self, state):
        """
        One hot encoding for the state.
        Each field input of 3x3 matrix will be displayed with 0 (blank), 1 (player 1), -1 (player 2)
        :param state: state to encode
        :return: encoded state
        """
        pass

    def __choose_optimal_move(self, state) -> int:
        """
        Choose optimal move based on the calculated and best predicted values of current state.
        :param state: current state
        :return: best move with highest value (randomly select for equal values)
        """
        v = -float('Inf') # most negative value (negative infinity float)
        v_list = [] # list of all calculated values
        idx = [] # move index for chosen move
        for move in Board.POSSIBLE_ACTIONS:
            value = self.__calc_value(state, move)
            v_list.append(round(float(value), 5))

            if value > v:
                v = value
                idx = [move]
            elif v == value:
                idx.append(move)

        idx = random.choice(idx)
        return idx

    def __calc_value(self, state, move) -> float:
        """
        Calculate a tensor and predict the reward
        :param state: current state
        :param move: current move
        :return: most predicted value (predicted reward)
        """

        tensor = self.state_to_tensor(state, move)
        value = self.model.predict(tensor)
        return value

    def __calc_target(self, prev_state, prev_move, state, reward):
        """
        Calculate the target vector (q value or reward)
        :param prev_state: previous state
        :param prev_move: previous move
        :param state: current state
        :param reward: previous reward
        :return: calculated target value
        """
        qvalue = self.__calc_value(prev_state, prev_move)
        v = []
        tensor = self.state_to_tensor(prev_state, prev_move)

        for move in range(len(tensor[:,0][0])):
            v.append(self.__calc_value(state, move))

        if reward == 0:
            v_s_tag = self.gamma * np.max(v)
            target = np.array(qvalue + self.alpha * (reward + v_s_tag - qvalue))
        else:
            target = reward

        return target

    def __save_model(self):
        """
        save model as h5 file
        """
        tag = '_first' if self.tag == 1 else '_second'

        if self.online:
            tag += '_online'

        s = 'model_values' + tag + '.h5'

        try:
            os.remove(s)
        except:
            pass

        self.model.save(s)

    def predict_model(self, state, online: bool):
        if online:
            state = self.one_hot_encode_state(state)
            target = int(np.argmax(self.model.predict(state)))
        else:
            target = self.__choose_optimal_move(state)

        return target

    def __learn_batch(self, memory):
        """
        Learn model with a batch of states and actions from memory
        :param memory: saved states, actions and rewards
        """
        print('start learning player', self.tag)
        print('data length:', len(memory))

        # build x_train
        ind = 0
        x_train = np.zeros((len(memory), 2, 9))
        for v in memory:
            [prev_state, prev_move, _, _] = v
            sample = self.state_to_tensor(prev_state, prev_move)
            x_train[ind, :, :] = sample
            ind += 1

        # train with planning
        loss = 20
        count = 0
        while loss > 0.02 and count < 10:
            y_train = self.__create_targets(memory)
            history = self.model.fit(x_train, y_train, epochs=5, batch_size=256, verbose=0)
            self.history.append(history.history)
            loss = self.model.evaluate(x_train, y_train, batch_size=256, verbose=0)[0]
            count += 1
            print('planning number:', count, 'loss', loss)

        loss = self.model.evaluate(x_train, y_train, batch_size=256, verbose=0)
        print('player:', self.tag, loss, 'loops', count)

        self.__save_model()

    def train_model_offline(self, prev_state, prev_move, state, reward):

        self.__load_to_memory(prev_state, prev_move, state, reward)
        self.count_memory += 1

        if self.count_memory == self.batch_size:
            self.count_memory = 0
            self.__learn_batch(self.memory)
            self.memory = []

    def __load_to_memory(self, prev_state, prev_move, state, reward):
        """
        Load all q related things into memory to learn in batch
        :param prev_state: previous known state
        :param prev_move: previous made move
        :param state: new state
        :param reward: previous reward
        """
        self.memory.append([prev_state, prev_move, state, reward])

    def __create_targets(self, memory):
        """
        Create target vector for each state-action-pair in memory
        :param memory: saved states, actions and rewards
        :return: target vector
        """
        y_train_ = np.zeros((len(memory), 1))
        count_ = 0
        for v_ in memory:
            [prev_state_, prev_move_, state_, reward_] = v_
            target = self.__calc_target(prev_state_, prev_move_, state_, reward_)
            y_train_[count_, :] = target
            count_ += 1

        return y_train_

    def train_model_online(self, prev_state, prev_move,new_state, reward, discount_factor=0.8):
        """
        Train the model based on the current state, action and received reward
        :param prev_state: previous state
        :param new_state: new state
        :param prev_move: previous move
        :param discount_factor: discount factor of this q learner
        :param reward: received reward after prev action
        """
        prev_state = self.one_hot_encode_state(prev_state)
        new_state = self.one_hot_encode_state(new_state)

        target = reward + discount_factor * np.max( self.model.predict(new_state))
        target_vector = self.model.predict(prev_state)[0]
        target_vector[prev_move] = target
        history = self.model.fit(prev_state, target_vector.reshape(-1, len(Board.POSSIBLE_ACTIONS)), epochs=1, verbose=0)
        if self.history is not None:
            self.history = self.history.append(history.history)
        else:
            self.history = [history.history]

## Spezialisiertes Modell für eine TicTacToe Spiel

Hier wird eine spezialisierte Modellvariante für ein TicTacToe Spiel beschrieben.
Dieses Modell besitzt eine vielzahl an tiefen Schichten, damit ein möglichst geringer Loss Wert erreicht werden kann.
Die Schichten sind für den Observations und Aktionsraum eines TicTacToe Spiels angepasst (also jeweils die Größe 9).

![](../../../docs/qnn_model.png)

Für die Umsetzung des neuronalen Netzes haben wir uns stark an dem Prinzip des Q-Algorithmus gehalten. Das Prinzip der temporalen Verbindung und der Aktion-Zustand-Paare sind mit in die Entscheidung über den Aufbau des Netzes eingeflossen.

Die obere Abbildung zeigt das von uns ausgewählte Modell des Netzes. Es gibt genau zwei Eingangsvektoren, ein Vektor um den aktuellen Zustand abzubilden und einer für die ausgewählte Aktion. Die Eingangswerte werden zuvor noch vorverarbeitet, sodass z.B. der Zustand des 3x3 Bretts und die Aktion eindimensionell dargestellt werden. Die Werte werden zudem noch über ein One-Hot-Encoding für die Aktivierungsfunktionen der Neuronen vereinfacht. Im Fall des Spielbretts wird ein Spieler mit einer positiven 1 versehen und der Gegner mit einer -1. Leere Felder werden mit 0 initialisiert. Damit die ausgewählte Aktion genau einem Feld der neun verfügbaren zugeschrieben werden kann, wird die Aktion mittels One-Hot als 1 dargestellt und die nicht ausgewählten Felder mit einer 0 versehen. Der Eingangsvektor hat somit folgendes Format:

1. Vektor (Zustand): [0, 0, 0, 1, 0, -1, 1, 0, -1]
2. Vektor (Aktion): [1, 0, 0, 0, 0, 0, 0, 0, 0]

In diesem Beispiel sah das aktuelle Feld folgendermaßen aus:
&ensp;

|   |   |   |
|---|---|---|
| -  | - | - |
| x  | - | o |
| x  | - | o |

Und nach der ausgewählten Aktion:
&ensp;

|   |   |   |
|---|---|---|
| x  | - | - |
| x  | - | o |
| x  | - | o |

Letzendlich wird ein Array der Größe 2x9 als Eingangsvektor genutzt. Um diesen im Netz jedoch noch mehr zu vereinfachen wird in der ersten Schicht des Netzes der zweidimensionale Eingangsvektor in einen 1x18 Vektor transformiert. Damit erhält der obige Array folgendes Format:

[0, 0, 0, 1, 0, -1, 1, 0, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

Danach werden 9 Dense Layers als Hidden Layer angefügt. Die Anzahl ergab sich aus der Feldgröße des Bretts. Diese werden schließlich mit der ReLU Aktivierungsfunkion (rectified linear unit) versehen (s. Funktion unten).

$$f(x) = max(0,x)$$

Die ReLU Aktivierungsfunktion nochmal als Graph dargestellt:

![](../../../docs/relu.png)

Die Entscheidung auf ReLU fiel deshalb, da wir während des Spielens die Züge aufzeichnen und ab einer gewissen Größe direkt an des Netz zum Lernen weitergeben. Damit wollten wir ein Online Lernen ermöglichen, indem das Netz die eben geführten Züge und erhaltenen Belohnungen direkt anlernt. Dabei entsteht eine sehr hohe Rechenlast, wenn neue Daten erneut gelernt werden müssen. Um diese Last zu vermeiden, haben wir uns dazu entschieden das Netz schmal und schnell zu machen. Gewichtungen werden somit schnell bestimmt und Ergebnisse aus einem Lernzyklus schnell ausgegeben. Der Nachteil dabei ist die geringe Genauigkeit. Aus diesem Grund muss der Lerndurchgang mehrere Male druchgelaufen werden, bis ein möglichst geringer Verlustwert erreicht wurde.

Der Ausgabevektor ist schließlich nur noch ein einziger Wert, nämlich der eigentliche Q-Wert für das angegebene Aktion-Zustand-Paar. Da wir festgestellt haben, dass der Q-Algorithmus eine schnelle Gewinnstrategie mittels hoher Belohnungswerte lernt, dachten wir uns, dass das neuronale Netz verhersagen muss ob gutes oder eher schlechtes Feedback erwartet wird. Demnach wird beim fitten des Modells als Zielvektor für das Training die jeweilige Belohnung angegeben. Im folgendem Beispiel wird dies besser verdeutlicht:

* Der Agent sieht einen aktuellen Zustand
* Eine beste Aktion wird mittels Vorhersage aus dem verfügbaren Modell ausgesucht oder eine zufällige Aktion
* Die Umgebung gibt dem Agenten Feedback (als Belohnung oder Bestrafung)
* der letzte Zustand und die ausgewählte Aktion wird nun als Eingabevektor für das Training zusammengestellt
* die erlangene Belohnung/Bestrafung wird als Zielvektor bestimmt
* der Agent lernt nun, dass diese Aktion auf dem letzten Zustand zu einer Belohnung oder Bestrafung geführt hat

Der Hintergedanke dabei ist, dass das neuronale Netz eine feste Bindung zwischen Aktion-Zustand-Paar und Feedback aufbaut, in der Hoffnung, dass gleichartige neue Zustände mit ähnlichen Feedback bestimmt werden können. Damit müssen in der Theorie nicht mehr alle Zustände erlernt werden, sondern nur noch jene die strukturelle Unterschiede vorweisen. Die restlichen können mittels Schätzungen möglichst genau beschrieben werden, ähnlich nach dem Prinzip der linearen Regression.


In [ ]:
class TicTacToeModel(Model):
    """
    Special model for tic tac toe games.

    Consists of 2x9 input vector, dense network of 9 layers and a 1 sized target vector.
    Input vector consists an array with length 9 for the chosen move and an array for the state.
    Target vector consists of one value for the q value.
    """

    def __init__(self, tag, online=False):
        self.online = online
        super().__init__(tag, self.online)
        pass

    def create_model(self):
        """
        Creates keras model
        :return: keras model
        """
        print('new model')

        model = Km.Sequential()
        model.add(kl.Flatten(input_shape=(2, 9)))
        model.add(kl.Dense(18))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(18))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(18))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(18))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(18))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(18))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(18))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(9))
        model.add(kl.LeakyReLU(alpha=0.3))
        model.add(kl.Dense(1, activation='linear'))

        model.compile(optimizer='Adam', loss='mean_absolute_error', metrics=['accuracy'])

        model.summary()

        return model

    def state_to_tensor(self, state, move):
        """
        Generates a tensor of state and move index
        :param state: current state
        :param move: current move
        :return: tensor (2 dim array)
        """
        state = self.one_hot_encode_state(state)

        a = np.zeros(9).astype('float32')
        a[move] = 1 # one hot encoding for chosen action (1 for the chosen action an 0 for none)

        state = np.asarray(state).astype('float32')
        tensor = np.array((a, state))
        tensor = tensor.reshape((1, 2, 9))

        return tensor

    def one_hot_encode_state(self, state):
        """
        One hot encoding for the state.
        Each field input of 3x3 matrix will be displayed with 0 (blank), 1 (player 1), -1 (player 2)
        :param state: state to encode
        :return: encoded state
        """
        state = state.flatten() # flatten 3x3 matrix because of 1 length input vector for state

        for i in range(len(state)):
            if state[i] is None:
                state[i] = 0
            if state[i] == 'x':
                state[i] = 1
            if state[i] == 'o':
                state[i] = -1

        state = np.asarray(state).astype('float32')
        state = state.reshape((1, 9))

        return state

## Schmale Modellvariante eines TicTacToe Spiels

Da wir auch ein online Lernverfahren probieren wollten und das letzte Modell dafür zu vielschichtig war, musste eine schmale Variante entwickelt werden. Diese Variante baut nur noch auf eine tiefe Schicht auf.

Als Eingangsvektor wird diesmal nur der eindimensionale Zustandsraum (Länge 9) übermittelt. Das Ziel des kurzfristigen Lernens soll dieses Mal direkt die Ausgabe von 9 möglichen Q-Werten für jede mögliche Aktion von diesem Zustand aus sein. Damit wird die Anzahl an Lerndurchgängen deutlich verringert, da pro Zustand direkt der Trainingsvorgang gestartet wird.

Auch in diesem Modell wird die ReLU Aktivierungsfunktion einegsetzt, sowie der Adam Optimierungsalgorithmus.

Das entstandene Modell sieht demnach wie folgt aus:

![](../../../docs/qnn_small_model.png)

In [ ]:
class TicTacToeModelSmall(TicTacToeModel):
    """
    Special model for tic tac toe games.

    Consists of 1x9 input vector, dense network of 1 layer and a 9 sized target vector.
    Input vector consists an array with length 9 for the state.
    Target vector consists of 9 sized array for 9 possible rewards (one for each action).
    """

    def __init__(self, tag, observation_space=9, action_space=9, online=False):
        self.observation_space = observation_space
        self.action_space = action_space
        self.online = online
        super().__init__(tag, self.online)
        pass

    def create_model(self):
        """
        Creates keras model
        :return: keras model
        """
        print('new model')

        model = Km.Sequential()
        model.add(kl.InputLayer(batch_input_shape=(1, self.observation_space)))
        model.add(kl.Dense(20, activation='relu'))
        model.add(kl.Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer='adam', metrics=['mae'])

        model.summary()

        return model